In [52]:
import numpy as np

import pandas as pd

import time

import lightgbm as lgb

from lightgbm import LGBMRegressor

from sklearn.datasets import make_regression

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [53]:
# Generate a large dataset

X, y = make_regression(n_samples=1_000_000, n_features=50, noise=0.1, random_state=42)


In [54]:
# Split into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [55]:
print(f"Dataset Size \n {X_train.shape[0]} Training samples \n {X_test.shape[0]} Testing samples" )

Dataset Size 
 800000 Training samples 
 200000 Testing samples


Converting to LightGBM's optimized lgb.Dataset format for memory efficiency.

In [56]:
train_data = lgb.Dataset(X_train ,label=y_train)

test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

Training LightGBM with Default CPU

Train the model without GPU acceleration first and measure the time taken.

In [57]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

Speeding Up LightGBM on Large Datasets

-------->  LightGBM uses histogram-based learning, which significantly reduces memory usage.

In [58]:
X_train = X_train.astype(np.float32)

X_test = X_test.astype(np.float32)

Enable Multi-threading

Utilize multiple CPU cores by setting:

In [59]:
params['num_threads'] = -1  # Use all available CPU cores


In [60]:
params['feature_fraction'] = 0.7  # Use 70% of features per iteration
params['bagging_fraction'] = 0.7  # Use 70% of data per iteration
params['bagging_freq'] = 5        # Perform bagging every 5 iterations

In [61]:
# Measure training time

start_time = time.time()

model = lgb.train(
    params, 
    train_data, 
    valid_sets=[test_data],
    num_boost_round=500, 
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),  
        lgb.log_evaluation(period=50)  
    ]
    )



end_time = time.time()

Training until validation scores don't improve for 50 rounds
[50]	valid_0's rmse: 53.7263
[100]	valid_0's rmse: 26.9184
[150]	valid_0's rmse: 15.2632
[200]	valid_0's rmse: 10.6937
[250]	valid_0's rmse: 8.96545
[300]	valid_0's rmse: 8.29143
[350]	valid_0's rmse: 7.99567
[400]	valid_0's rmse: 7.84269
[450]	valid_0's rmse: 7.7611
[500]	valid_0's rmse: 7.69598
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 7.69598


In [62]:
print(f"Training time CPU : {end_time-start_time : .2f}")

Training time CPU :  13.46


LightGBM Supports OPENCL Which is not compatible with macos ---> run this with cuda GPU

In [ ]:
params['device'] = 'gpu'
params['gpu_platform_id'] = 0
params['gpu_device_id'] = 0


In [ ]:
start_time_gpu = time.time()

model_gpu = lgb.train(
    params, 
    train_data, 
    valid_sets=[test_data],
    num_boost_round=500, 
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)  
    ]
    )

end_time_gpu = time.time()

print(f"Training Time (GPU): {end_time_gpu - start_time_gpu:.2f} seconds")


In [ ]:
y_pred = model.predict(X_test)


In [ ]:

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
